IMPORTING LIBRARIES

In [ ]:

 # Importing all the necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import string
import re
import nltk
from nltk.corpus import stopwords as Stopwords
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import Logistic Regression
from sklearn.metrics import classification_report, log_loss
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import nltk
nltk.download ("stopwords")

LOAD DATASET

In [ ]:
reviews_dataset = pd.read_csv("")
reviews_dataset.head(10)

In [ ]:
reviews_dataset.info()

DATA PREPROCESSING

In [ ]:
reviews_dataset.dropna(inplace=True)

In [ ]:
reviews_dataset.drop_duplicates(inplace=True)

In [ ]:
reviews_dataset.info()

CLEANING RATING COLUMN

In [ ]:
reviews_dataset

In [ ]:
reviews_dataset["Rating"] = reviews_dataset["Rating"].apply(lambda x : re.findall("\d",x)[0])

In [ ]:

1 # Mapping positive, neutral and negative to rating values
2
3 reviews_dataset["Sentiment"] = reviews_dataset ["Rating"].map({"5":"Positive", "4":"Positive", "3":"Neutral", "2":"Negative", "1":"Negative"}) 4 reviews_dataset

CREATING YEAR OF STAY COLUMN

In [ ]:

1 # Creating a new column year of stay from our existing column date of stay
3 reviews_dataset [ "Year of stay"] = reviews_dataset ["Date of stay"].apply(lambda x: "".join(re.findall ("\d\d\d\d", x))) 4 reviews_dataset

CREATING MONTH OF STAY COLUMN

In [ ]:

1# Creating a month of stay column from date of stay column
2 reviews dataset["Month of stay"] reviews_dataset["Date of stay"].apply(lambda x: ".join(re.findall(r' (Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:11)?|May Jun(?:e)?|July)?|A 3 reviews dataset

DROPPING ROWS

In [ ]:

1 # Dropping all rows for which year is less than 2018
2
3 reviews_dataset.drop(reviews_dataset [reviews_dataset [ "Year of stay"] < '2018']. index, axis=0, inplace=True)
4
5
6
7 # Dropping all neutral sentiment rows
8
9 reviews_dataset.drop(reviews_dataset [reviews_dataset ["Sentiment"]
==
"Neutral"].index, axis=0, inplace=True)

In [ ]:

1 # Checking information for the dataset after removing rows
2 #After removing reviews before 2018 and neutral sentiment, remaining row number is 8293.
3
4 reviews_dataset.info()

CHECKING PERCENTAGE OF SENTIMENT

In [ ]:

1 # Checking percentage of each sentiment in our dataset
2
3 reviews_dataset.Sentiment.value_counts (normalize=True)

In [ ]:

1 # Shuffling the dataset
2
3 reviews_dataset = reviews_dataset.sample(frac = 1)
4 reviews_dataset

FUNCTION TO PREPROCESS THE REVIEW COLUMN

In [ ]:

1 def preprocess_data(text):
Returns text after removing numbers, punctuations, urls, emojis, html tags and lowercasing all the words in given text.
# removing numbers
text = re.sub(r'[^\w\s]',, str(text)) # removing punctuations text="".join(x.lower() for x in text.split()) # lower casing the text
text = re.sub(r'https?://\S+|www\.\S+", ", text) # removing urls text = re.sub(r'<."?>', '', text) # removing html tags
emoji_pattern= re.compile("["

text = re.sub(r'[0-9]+', '', str(text))

text = emoji_pattern.sub(r", text)
u"\U0001F600-\U0001F64F" # emoticons
u"\U0001F300-\U0001F5FF" # symbols & pictographs u"\U0001F680-\U0001F6FF" # transport & map symbols u"\U0001F1E0-\U0001F1FF" # flags (ios)
u"\U00002702-\U00002780"
u"\U000024C2-\U0001F251"
"]+", flags=re.UNICODE)
text = ".join(x for x in text.split() if x not in stopwords.words('english')) return text

In [ ]:

1 # Applying preprocess_data function to review column
2
3 reviews_dataset [ "Preprocessed_reviews"] = reviews_dataset [ "Review"].apply(lambda x: preprocess_data(x)) 4 reviews_dataset

CREATING A COLUMN FOR NUMBER OF WORDS IN REVIEW

In [ ]:

1 # Word count
2
3 reviews_dataset ["Word_count"]
4 reviews_dataset.head()
=
reviews_dataset ["Review"].apply(lambda x: len(str(x).split()))

CREATING A COLUMN FOR NUMBER OF UNIQUE WORDS IN REVIEW

In [ ]:

1 # unique Word count
2
3 reviews_dataset ["unique_Word_count"] = reviews_dataset ["Review"].apply(lambda x: len(set(str(x).split()))
4 reviews_dataset.head()


CREATING A COLUMN FOR NUMBER OF STOPWORDS IN REVIEW

In [ ]:
 # Number of stop words in review

 stop_words = Stopwords.words ('english')
 reviews_dataset["Stopword_count"] = reviews_dataset ["Review"].apply(lambda x: len([w for w in str(x).lower().split() if w not in stop words])) 5 reviews_dataset.head()

CREATING A COLUMN FOR AVERAGE WORD LENGTH IN A REVIEW

In [ ]:
# mean word length

reviews_dataset["Mean_word_length"] = reviews_dataset["Review"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
reviews_dataset.head()

CREATING A COLUMN FOR NUMBER OF CHARACTER IN A REVIEW

In [ ]:
# char count

reviews_dataset["char_count"] = reviews_dataset["Review"].apply(lambda x: len(str(x)))
reviews_dataset.head()

CREATING A COLUMN FOR NUMBER OF PUNCTUATION IN A REVIEW

In [ ]:
# punctuation count

reviews_dataset["punctuation_count"] = reviews_dataset["Review"].apply(lambda x: len([p for p in str(x) if p in string.punctuation]))
reviews_dataset.head()

##EXPLORATORY DATA ANALYSIS (EDA)

PIE CHART

In [ ]:
# creating a pie chart showing target distribution in dataset

plt.figure(figsize=(9, 9))
reviews_dataset['Sentiment'].value_counts().plot(kind='pie', autopct="%1.1f%%", startangle=90, fontsize=14, colors=['#27EEA4', '#F37D67']);
s.set_ylabel('');
s.set_title('Target Distribution in dataset', fontsize=20);

N-GRAMS

In [ ]:
# Function for generating ngrams

def generate_ngrams(text, n_gram=1):
    """
    Returns ngrams for the given text.

    Parameters:
    text : for which we want ngrams
    n_gram : value for ngram
    """
    token = [token for token in text.lower().split(' ') if token != '' if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

UNI-GRAMS

In [ ]:
# Creating a default dictionary for positive and negative unigrams
positive_unigrams = defaultdict(int)
negative_unigrams = defaultdict(int)

# Number of ngrams we want
N = 25

# Loop for updating value of positive_unigrams
for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment'] == "Positive"]:
    for word in generate_ngrams(review):
        positive_unigrams[word] += 1

# Loop for updating value of negative_unigrams
for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment'] == "Negative"]:
    for word in generate_ngrams(review):
        negative_unigrams[word] += 1

# Creating dataframes using default dictionaries
df_positive_unigrams = pd.DataFrame(sorted(positive_unigrams.items(), key=lambda x: x[1], reverse=True))
df_negative_unigrams = pd.DataFrame(sorted(negative_unigrams.items(), key=lambda x: x[1], reverse=True))

fig, axes = plt.subplots(ncols=2, figsize=(24, 10))
plt.tight_layout(pad=4.0)

# Plotting positive and negative unigrams dataset
sns.despine()
sns.barplot(y=df_positive_unigrams[0].values[:N], x=df_positive_unigrams[1].values[:N], ax=axes[0], color='#272EAA')
sns.barplot(y=df_negative_unigrams[0].values[:N], x=df_negative_unigrams[1].values[:N], ax=axes[1], color='#F37D67')

In [ ]:

axes[0].set_title(f'Top {N} most common unigrams in Positive Reviews', fontsize=15)
axes[1].set_title(f'Top {N} most common unigrams in Negative Reviews', fontsize=15)
plt.show()

BI-GRAMS

In [ ]:
# Creating a default dictionary for positive and negative bigrams
positive_bigrams = defaultdict(int)
negative_bigrams = defaultdict(int)

# Number of ngrams we want
N = 25

# Loop for updating values of bigrams in default dictionary
for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment']=="Positive"]:
    for word in generate_ngrams(review, n_gram=2):
        positive_bigrams[word] += 1

for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment']=="Negative"]:
    for word in generate_ngrams(review, n_gram=2):
        negative_bigrams[word] += 1

# Creating a dataset using default dictionaries
df_positive_bigrams = pd.DataFrame(sorted(positive_bigrams.items(), key=lambda x: x[1], reverse=True))
df_negative_bigrams = pd.DataFrame(sorted(negative_bigrams.items(), key=lambda x: x[1], reverse=True))

fig, axes = plt.subplots(ncols=2, figsize=(24, 10))
plt.tight_layout(pad=4)

# Plotting both positive and negative bigrams
sns.despine()
sns.barplot(x=df_positive_bigrams[1].values[:N], y=df_positive_bigrams[0].values[:N], ax=axes[0], color='#72EEA4')
sns.barplot(x=df_negative_bigrams[1].values[:N], y=df_negative_bigrams[0].values[:N], ax=axes[1], color='#F37D67')

axes[0].set_title(f'Top {N} most common bigrams in Positive Reviews', fontsize=15)

In [ ]:
axes[0].set_title(f'Top {N} most common bigrams in Positive Reviews', fontsize=15)
axes[1].set_title(f'Top {N} most common bigrams in Negative Reviews', fontsize=15)
plt.show()

TRI-GRAMS

In [ ]:
# Creating default dictionary for positive and negative trigrams
positive_trigrams = defaultdict(int)
negative_trigrams = defaultdict(int)

# Number of ngrams we want
N = 25

# Loop for updating default dictionaries
for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment']=="Positive"]:
    for word in generate_ngrams(review, n_gram=3):
        positive_trigrams[word] += 1

for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment']=="Negative"]:
    for word in generate_ngrams(review, n_gram=3):
        negative_trigrams[word] += 1

# Creating a dataframe using dictionaries
df_positive_trigrams = pd.DataFrame(sorted(positive_trigrams.items(), key=lambda x: x[1], reverse=True))
df_negative_trigrams = pd.DataFrame(sorted(negative_trigrams.items(), key=lambda x: x[1], reverse=True))

fig, axes = plt.subplots(ncols=2, figsize=(24, 10))
plt.tight_layout(pad=4.0)

# Plotting positive and negative trigrams
sns.despine()
sns.barplot(y = df_positive_trigrams[0].values[:N], x = df_positive_trigrams[1].values[:N], ax=axes[0], color="#72EEA4")
sns.barplot(y = df_negative_trigrams[0].values[:N], x = df_negative_trigrams[1].values[:N], ax=axes[1], color="#F37D67")
axes[0].set_title('Top N most common trigrams in Positive Reviews', fontsize=15)

In [ ]:
axes[0].set_title(f'Top {N} most common trigrams in Positive Reviews', fontsize=15)
axes[1].set_title(f'Top {N} most common trigrams in Negative Reviews', fontsize=15)
plt.show()

BAR PLOT FOR EACH YEAR

In [ ]:
# Plotting number of reviews for each year

plt.figure(figsize = (24, 5))
years = ['2018', '2019', '2020', '2021']
sns.despine()
sns.countplot(x="Year of stay", data=reviews_dataset, order=years, color='#62DCEC');

PLOTTING WORD COUNT

In [ ]:
# 1  Plotting word count for reviews

fig, axes = plt.subplots(ncols = 2, figsize=(24, 6))

sns.despine()
sns.histplot(ax=axes[0], x="word_count", data=reviews_dataset[reviews_dataset['Sentiment'] == 'Positive'], kde=True, color="#172EE4")
sns.histplot(ax=axes[1], x="word_count", data=reviews_dataset[reviews_dataset['Sentiment'] == 'Negative'], kde=True, color="#37067")

axes[0].set_title("Positive reviews", fontsize=16)
axes[1].set_title("Negative reviews", fontsize=16)

axes[0].set_ylabel("Number of reviews", fontsize=12)
axes[1].set_ylabel("Number of reviews", fontsize=12)
axes[0].set_xlabel("word count in review", fontsize=12)
axes[1].set_xlabel("word count in review", fontsize=12)

fig.suptitle("Word count for reviews", fontsize=20)
plt.show()

PLOTTING UNIQUE WORD COUNT

In [ ]:
# 1 # Plotting unique word count for reviews

fig, axes = plt.subplots(ncols=2, figsize=(24, 6))
sns despine()

sns.histplot(ax=axes[0], x="Unique_word_count", data=reviews_dataset[reviews_dataset['Sentiment']=='Positive'], kde=True, color="#72EAE4")
sns.histplot(ax=axes[1], x="Unique_word_count", data=reviews_dataset[reviews_dataset['Sentiment']=='Negative'], kde=True, color="#F37D67")

axes[0].set_title("Positive reviews", fontsize=16)
axes[1].set_title("Negative reviews", fontsize=16)

axes[0].set_ylabel("Number of reviews", fontsize=12)
axes[1].set_ylabel("Number of reviews", fontsize=12)
axes[0].set_xlabel("unique word count in review", fontsize=12)
axes[1].set_xlabel("Unique word count in review", fontsize=12)

fig.suptitle("Unique word count for reviews", fontsize=20)
plt.show()

PLOTTING STOP WORD COUNT

In [ ]:
# Plotting stopword count for reviews

fig, axes = plt.subplots(ncols=2, figsize=(24, 6))

sns.despine()
sns.histplot(ax=axes[0], x="Stopword_count", data=reviews_dataset[reviews_dataset['Sentiment']=='Positive'], kde=True, color="#27EAA4");
sns.histplot(ax=axes[1], x="Stopword_count", data=reviews_dataset[reviews_dataset['Sentiment']=='Negative'], kde=True, color="#F37D67");

axes[0].set_title("Positive reviews", fontsize=16)
axes[1].set_title("Negative reviews", fontsize=16)

axes[0].set_ylabel("Number of reviews", fontsize=12)
axes[1].set_ylabel("Number of reviews", fontsize=12)
axes[0].set_xlabel("Stopword count in review", fontsize=12)
axes[1].set_xlabel("Stopword count in review", fontsize=12)

fig.suptitle("Stopword count for reviews", fontsize=20)
plt.show()

PLOTTING CHARACTER COUNT

In [ ]:
# 1 = Plotting character count for reviews
fig, axes = plt.subplots(ncols = 2, figsize=(24, 6))
sns.despine()
sns.histplot(ax=axes[0], x="Char_count", data=reviews_dataset[reviews_dataset['sentiment'] == 'Positive'], kde=True, color="#72EE44")
sns.histplot(ax=axes[1], x="Char_count", data=reviews_dataset[reviews_dataset['sentiment'] == 'Negative'], kde=True, color="#F37D67")

axes[0].set_title("Positive reviews", fontsize=16)
axes[1].set_title("Negative reviews", fontsize=16)

axes[0].set_ylabel("Number of reviews", fontsize=12)
axes[1].set_ylabel("Number of reviews", fontsize=12)
axes[0].set_xlabel("Character count in review", fontsize=12)
axes[1].set_xlabel("Character count in review", fontsize=12)

fig.suptitle("Character count for reviews", fontsize=20)
plt.show()

PLOTTING PUNCTUATION COUNT

In [ ]:
1  # Plotting punctuation count for reviews
2  fig, axes = plt.subplots(ncols = 2, figsize=(24, 6))
3  sns.despine()
4  sns.histplot(ax=axes[0], x="punctuation_count", data=reviews_dataset[reviews_dataset['Sentiment'] == 'positive'], kde=True, color="#72EEA4")
5  sns.histplot(ax=axes[1], x="punctuation_count", data=reviews_dataset[reviews_dataset['Sentiment'] == 'negative'], kde=True, color="#F37D67")
6  axes[0].set_title("Positive reviews", fontsize=16)
7  axes[1].set_title("Negative reviews", fontsize=16)
8  axes[0].set_xscale('log')
9  axes[0].set_ylabel("Number of reviews", fontsize=12)
10 axes[1].set_ylabel("Number of reviews", fontsize=12)
11 axes[0].set_xlabel("Punctuation count in review", fontsize=12)
12 axes[1].set_xlabel("Punctuation count in review", fontsize=12)
13 fig.suptitle("Punctuation count for reviews", fontsize=20)
14 plt.show()

##WORD CLOUD

WORD CLOUD FOR POSTIVE REVIEW

In [ ]:
# Plotting a wordcloud for positive reviews

positive = " ".join(review for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment'] == "Positive"])
wordcloud = WordCloud(background_color='white', stopwords=STOPWORDS, max_words=500, max_font_size=40, random_state=42, colormap='Greens').generate(positive)

plt.figure(figsize=(15, 10))
plt.title("Most common words in positive reviews")
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

WORD CLOUD FOR NEGATIVE REVIEW

In [ ]:
# Plotting wordcloud for negative reviews
negative = ' '.join(review for review in reviews_dataset['Preprocessed_reviews'][reviews_dataset['Sentiment'] == "negative"])
wordcloud = WordCloud(background_color='white', stopwords=STOPWORDS, max_words=500, max_font_size=40, random_state=42, colormap="Reds").generate(negative)
plt.figure(figsize=(15, 10))
plt.title("Most common words in negative reviews")
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Create custom stop words

all_text = " ".join(list(reviews_dataset["Preprocessed_reviews"]))
words = pd.Series(all_text.split(" "))
frequent_words = words.value_counts()[:20]

In [ ]:
# Using label encoder to encode sentiment column

le = LabelEncoder()
reviews_dataset['Sentiment'] = le.fit_transform(reviews_dataset['Sentiment'])

In [ ]:
x = reviews_dataset.iloc[:,0]
x

In [ ]:
y = reviews_dataset.iloc[:,3]
y

TRAINING AND TESTING

In [ ]:
# Splitting the dataset into training and testing dataset
xtrain, xtest, ytrain, ytest = train_test_split(x, y,
                                               random_state=42,
                                               test_size=0.2, shuffle=True)

TRAINING AND VALIDATION

In [ ]:
# Splitting the dataset into training and testing dataset
xtrain, xval, ytrain, yval = train_test_split(x, y,
                                               random_state=42,
                                               test_size=0.2, shuffle=True)

MODEL TRAINING

In [ ]:
# Initializing TfidfVectorizer
tfv = TfidfVectorizer(min_df=3, max_features=None, decode_error="replace", preprocessor=preprocess_data,
                      strip_accents='unicode', analyzer='word', token_pattern='w(1,)',
                      ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
                      stop_words='english')

# Fitting Tfidf vectorizer to training dataset
tfv.fit(list(xtrain))

# Transforming the training and validation dataset
xtrain_tfv = tfv.transform(xtrain)
xval_tfv = tfv.transform(xval)

In [ ]:
# 1 Initializing XGClassifier
xgb = XGClassifier(learning_rate=0.02, n_estimators=600, max_depth=4, subsample=1.0, min_child_weight=1, gamma=0.5, colsample_bytree=0.8, random_state=42)

# 3 Fitting xgb on training dataset
model = xgb.fit(train_tfv, ytrain)

# 7 Making predictions on validation dataset
predictions = model.predict_proba(val_tfv)
print(f"logloss: {log_loss(yval, predictions)}")

ypred_xg = model.predict(val_tfv)
print(classification_report(yval, ypred_xg))

MODEL EVALUATION

In [ ]:
# transforming test dataset using the same tfidf vectoririzer we fit on training dataset

xtest_tfv = tfv.transform(xtest)

In [ ]:
p = xgb.predict(xtest_tfv)
p

In [ ]:
#making prediction on testing dataset

predictions = xgb.predict_proba(xtest_tfv)
print("logloss : %0.3f " % log_loss(ytest,prediction))
ypred_xg = xgb.predict(xtest_tfv)
print(classification_report(ytest,ypred_xg))

ACCURACY

In [ ]:
accuracy = accuracy_score(ytest,ypred_xg)
print(accuracy)

MODEL PREDICTION

In [ ]:
text = [input("Enter a review : ")]
text_tfv = tfv.transform(text)

y_pred = model.predict(text_tfv)

#get the predicted sentiment
senti = y_pred[0]

#interpret the prediction
if senti > 0:
  print("the review is predicted to be positive")
else:
  print("the review is predicted to be negative")